## Save image when ray tracing completed

This example shows how to use `on_rt_accum_done` callback to save image when accumulation is finished. Tk GUI is used and the image is saved after each interactive action (rotation, etc), when accumulation arrives to the last frame.

In [1]:
import numpy as np
from plotoptix import TkOptiX
from plotoptix.materials import m_clear_glass, m_plastic

Callback executed when the last accumulation frame is completed:

In [2]:
def save_image(rt: TkOptiX) -> None:
    rt.save_image("teapot.png")
    print("rt completed!")

Setup the raytracer:

In [3]:
rt = TkOptiX(on_rt_accum_done=save_image, start_now=False)

In [4]:
rt.set_param(min_accumulation_step=2,      # update image every 2 frames
             max_accumulation_frames=20,   # accumulate 20 frames
             light_shading="Hard")         # best caustics
rt.set_uint("path_seg_range", 15, 30)      # allow many reflections/refractions

rt.setup_material("glass", m_clear_glass)
rt.setup_material("plastic", m_plastic)

rt.set_float("tonemap_exposure", 0.4)      # sRGB tonning and AI denoising
rt.set_float("tonemap_gamma", 2.2)
rt.set_float("denoiser_blend", 0.3)
rt.add_postproc("Denoiser")

rt.set_background(0)
rt.set_ambient(0)

Prepare a simple scene objects, camera and lights:

In [5]:
pos = [[1, 0, 0], [21, 0, 0], [-0.5, 0, 20], [20, 0, 21]]
v = [[0, 40, 0], [0, 35, 0], [0, 25, 0], [0, 5, 0]]
u=[19, 0, 0]; w=[0, 0, 19.5]
c = [[0.95, 0.95, 0.95], [0.1, 0.2, 0.3], [0.95, 0.95, 0.95], [0.95, 0.95, 0.95]]

In [6]:
rt.set_data("plane", geom="Parallelograms", pos=[[-100, -0.1, -100]], u=[200, 0, 0], v=[0, 0, 200], c=0.95)
rt.set_data("blocks", geom="Parallelepipeds", mat="plastic", pos=pos, u=u, v=v, w=w, c=c)
rt.set_data("ball", geom="ParticleSet", pos=[[19, 1.5, 43]], r=1.5, c=[0.7, 0.1, 0])
rt.load_merged_mesh_obj("../1_basic_plot_making/data/utah-teapot.obj", "teapot", mat="glass", c=[12, 9.5, 6])
rt.load_merged_mesh_obj("../1_basic_plot_making/data/wineglass.obj", "wineglass", mat="glass", c=10, make_normals=True)

In [7]:
rt.rotate_primitive("blocks", 3, [0, 0.02*np.pi, 0])

rt.scale_geometry("teapot", s=0.425)
rt.move_geometry("teapot", v=[26, 24, 30])
rt.rotate_geometry("teapot", [0.2*np.pi, 0.1*np.pi, -0.25*np.pi])

rt.scale_geometry("wineglass", s=1.8)
rt.move_geometry("wineglass", v=[32, 11, 32])
rt.rotate_geometry("wineglass", [0, 0.2*np.pi, 0.65*np.pi])

In [8]:
rt.setup_camera("cam1", cam_type="DoF", eye=[77, 34, 97], target=[20, 14, 23],
                aperture_radius=2.7, aperture_fract=0.3,
                fov=30, focal_scale=0.837)
rt.setup_light("light1", pos=[40, 65, 20], color=[15, 13, 11], radius=12)
rt.setup_light("light2", pos=[20, 30, 90], color=[15, 18, 20], radius=10)

Open Tk GUI window and start ray tracing. Message is printed when callback is executed, you can go for a coffee if your GPU is < $1k.

In [9]:
rt.start()

rt completed!


Close the ray tracing instance:

In [10]:
rt.close()